In [ ]:
# Description

In [ ]:
# -*- coding=utf-8 -*-
"""
주요 기능: GoogleAPI 연결, 기본 변수(file_id, mime_type) 설정
    - Super Class(GoogleSheets, GoogleDrive, ... 등) 
    - Google API Service 접속
        - user_nick(OAuth 2.0 Client ID)
        - bot_nick(Service Account)
        - api_key(API Key)  ## 구현되어 있지 않음
    - list: 파일/폴더 검색(file_type, query)
    - id_by_name: file_id 검색(app_name, file_name)
    - id_by_path: file_id 검색(app_name, file_path)

사용 방법: 
    - 

참조 자료:
    - 

필요 환경:
    - pip requirements:
    - 
"""

# References
- [Google Drive for Developers > Drive API(V3)](https://developers.google.com/drive/api/v3/reference)
- [Google Drive for Developers > Drive API(V3) > Files](https://developers.google.com/drive/api/v3/reference/files)

# Import Packages

## BuiltIn Packages

In [ ]:
##@@@ Package/Module
##============================================================

##@@ Built-In Package/Module
##------------------------------------------------------------
import os, sys
from pprint import pprint

import json

from googleapiclient.discovery import build

## External Packages(installed by pip)

In [ ]:
## Local(PC Drive) 환경인지 Cloud(Google Drive) 환경인지 확인
ENV = "_LOCAL_"
if "content" in os.getcwd():
    ENV = "_GDRIVE_"

if ENV == "_LOCAL_":
    try: # LOCAL SHELL
        cwd = __file__
    except:  # LODAL JUPYTER
        import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈
    
    from GoogleAuth import GoogleAuth
    PATH_CONFIGS = "./configs"
else:
    sys.path.insert(0, '/content/drive/MyDrive/__COLAB/PACKAGES')
    import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈
    sys.path.insert(0, '/content/drive/MyDrive/__COLAB/__GOOGLE')
    ## NOTE: 현재 디렉토리로 이동한 후 import해야 함
    # %cd /content/drive/MyDrive/__COLAB/__GOOGLE
    from GoogleAuth import GoogleAuth
    PATH_CONFIGS = "/content/drive/MyDrive/__COLAB/CONFIGS/GOOGLE/"

# Global Constants

In [ ]:
##@@ User Package/Module
##------------------------------------------------------------

##@@@ Constant/Varible
##============================================================

##@@ Constant
##------------------------------------------------------------

##@@ Variable(Golobal)
##------------------------------------------------------------

# Private Functions

## mime_type

In [ ]:
##@@@ Private Function
##============================================================

##@@ path, credentials
##------------------------------------------------------------
def _get_path(path_type, user_nick="", settings_folder=PATH_CONFIGS):
    """

    """
    path = settings_folder[:-1] if settings_folder[-1] == "/" else settings_folder
    if path_type == 'mime_type':  # api spec(version, scope)
        path += f"/__mime_type.json"
    else:  ## TODO: 환경 변수(export_map 등) 설정 추가
        pass

    return path


def _set_map_mime_type():
    with open(_get_path("mime_type"), 'r') as f:
        return json.load(f)

# Main Class

In [ ]:
##@@@ Main Class
##============================================================
class GoogleAPI(GoogleAuth):
    def __init__(self, api_name, user_nick=None, bot_nick=None, path=None):
        """__init__: Object 초기화
        Desc:
            - Google Drive Service 연결(user_nick)
            - path 폴더 기준

        Args:
            - user_nick(str, None): 구글계정 별칭(master: monblue@snu.ac.kr / moonitdev: moonitdev@gmail.com / ...)
            - bot_nick(str, None): GCP(google cloud platform) Project 서비스 계정(Bot) 별칭(mats, moonMaster, ...)
            - path(str, None): Google Drive 기준 폴더(경로) 예) __COLAB/_TEST

        Usages:
            - [description]
        """
        super().__init__(user_nick=user_nick, bot_nick=bot_nick)
        api_name = api_name.lower()  # TODO: 필요한지 확인
        self.api_name = api_name
        self.user_nick = user_nick
        self.bot_nick = bot_nick
        self.mime_type = _set_map_mime_type()
        self.service = build(api_name, self.version[api_name], credentials = self.credentials)
        
        if api_name == "drive":
            self.drive = self.service 
        else:
            self.drive = build("drive", self.version["drive"], credentials = self.credentials)


    ##@@ Private Function
    ##------------------------------------------------------------
    @staticmethod
    def _path_split(path, separator="/"):
        path = path.replace("\\", "/")
        return path if type(path) == list else path.split(separator) if type(path) == str else None ## NOTE: path를 list 형식으로 변경


    def _query_mime_type(self, type=None):
        if type == None:
            return None
        elif type == 'file':
            query = "mimeType != 'application/vnd.google-apps.folder'"
        else:
            query = f"mimeType = '{self.mime_type[type]}'"
        return query


    def _query(self, type=None, queries=[]):
        return " and ".join(queries if type is None else [self._query_mime_type(type)] + queries)


    def _list(self, type=None, queries=[], trashed=False, out_type="dict"):
        """list_of_book: google sheets 전체 목록
        Desc:
            - type(str, None): app_name / file type_nick  예) sheets, script, xlsx, json, ...
            - queries(list, []): 검색 쿼리 리스트: ["name = 'file_name1'", ...]
            - trashed(bool, False): 휴지통에 있는 파일 검색 여부 

        Returns:
            - list: file type, queries에 해당하는 folder/file 리스트
        """
        # print(f"mimetype: {_APP_NAME_MAP[app]['mimeType']}")
        queries = queries + ["trashed = false"] if trashed == False else queries + ["trashed = true"]
        query = self._query(type=type, queries=queries)

        results = self.drive.files().list(
            q = query,
            fields = "nextPageToken, files(id, name, mimeType, parents)"
        ).execute()

        if out_type == "dict":
            results = {item['name']: item['id'] for item in results.get('files', [])}
        
        return results


    def _id_by_name(self, name, type=None, queries=[], trashed=False):
        queries = [f"name = '{name}'"] + queries
        files = self._list(type=type, queries=queries, trashed=trashed, out_type="dict")
        return files[name] if files else None


    def _children_by_parents_id(self, parents_id, folder=False):
        query = f"'{parents_id}' in parents"
        if folder:
            query += " and mimeType = 'application/vnd.google-apps.folder'"

        return self.drive.files().list(
            q = query, 
            fields="nextPageToken, files(id, name)"
        ).execute().get('files', [])


    def _child_id_by_name(self, parents_id, child_name):
        child = [d for d in self._children_by_parents_id(parents_id) if d.get("name", []) == child_name]
        return child[0]['id'] if len(child) > 0 else None


    def _parents_id_by_file_id(self, file_id):
        response = self.drive.files().get(fileId=file_id, fields="parents").execute()
        if response:
            return response["parents"][0]
        else:
            return "root"


    def _ids_by_path(self, path=None, separator="/", folder=False):
        """
        return: [{'': 'root'}, {'folder1': 'folder_id1'}, ... {'folder_i': None},... {'folder_n': None}]
        ## TODO: path에 파일이름이 포함되어 있는 경우, 폴더만 출력하기(folder=True)
        """
        path = path.strip()
        path = path[:-1] if path[-1] == "/" else path  ## TODO: 마지막 문자가 "///" 형태인 경우도 적용되도록
        names = self._path_split(path, separator=separator) ## NOTE: path를 list 형식으로 변경
        if not names:
            return "root"
        if folder:  # 마지막 경로 이름 제거(file로 간주) 
            names = names[:-1]

        paths = [{name: None} for name in names]
        print(f"paths: {paths}")
        file_id = "root" if names[0].lower() in ["", "내 드라이브", "my drive", "mydrive", "drive"] else self._id_by_name(names[0])
        paths[0] = {names[0]: file_id}
        for i, name in enumerate(names[:-1]):
            # print(f"BEFORE: name: {name}, file_id: {file_id}")
            _file_id = self._child_id_by_name(file_id, names[i+1])
            paths[i+1] = {names[i+1]: _file_id}
            # print(f"AFTER: names[{i+1}]: {names[i+1]}, file_id: {_file_id}, paths[{i+1}]: {paths[i+1]}")
            if _file_id == None:
                # paths[i+1] = {names[i+1]: file_id}
                break
            file_id = _file_id

        return paths


    def _create_folder_by_parents_id(self, name, parents_id=None):
        parents_id = self.id if parents_id == None else parents_id
        return self.drive.files().create(
            body = {
                'name': name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parents_id]
            },
            fields = 'id'
        ).execute().get('id', None)


    def _create_folders_by_paths(self, paths):
        """
        paths: [{'': 'root'}, {'folder1': 'folder_id1'}, ... {'folder_i': None},... {'folder_n': None}]
        """
        folder_names = [list(path.keys())[0] for path in paths]
        folder_ids = [list(path.values())[0] for path in paths]

        for i, folder_id in enumerate(folder_ids):
            if folder_id == None:
                # print(folder_names[i], folder_ids[i-1])
                folder_ids[i] = self._create_folder_by_parents_id(folder_names[i], parents_id=folder_ids[i-1])
        
        return folder_ids[-1]


    def _id_by_path(self, path=None, separator="/", folder=False, auto_create=False, parents=False):
        ## @@@ TODO: len(paths) == 0, len(paths) == 1인 경우 처리
        """
        return: [{'': 'root'}, {'folder1': 'folder_id1'}, ... {'folder_i': None},... {'folder_n': None}]
        ## TODO: path에 파일이름이 포함되어 있는 경우, 폴더만 출력하기(folder=True)
        auto_create: 해당 경로가 없으면 폴더 생성
        """
        paths = self._ids_by_path(path=path, folder=folder)
        last_name = list(paths[-1].keys())[0]  # 마지막 path 항목 file name
        last_id = list(paths[-1].values())[0]  # 마지막 path 항목 file_id(파일일 수 있음)
        parents_id = list(paths[-2].values())[0]  # 끝으로 2번째 path 항목 file_id(last_id가 파일인 경우에 사용)

        if auto_create:  # 폴더 자동 생성
            parents_id = self._create_folders_by_paths(paths)  # 마지막 생성된 folder id(마지막 항목이 파일인 경우: 위의 parents_id와 동일)
            ## folder 생성
        if parents:  # NOTE: (name, parents_id)
            if folder:
                parents_id = last_id  ## 폴더만 있는 경우
            return (last_name, parents_id)
        else:
            return id


    ##@@ Public Function
    ##------------------------------------------------------------
    def list(self, out_type="list"):
        return self._list(type=self.api_name, out_type=out_type)


    # def id_by_name(self, name, app="sheets"):
    def id_by_name(self, name):
        return self._id_by_name(name, type=self.api_name)


    def name_by_id(self, file_id):
        return self.drive.files().get(fileId=file_id, fields="name").execute()["name"]


    def id_by_path(self, path=None, separator="/"):
        path = _path_split(path) ## NOTE: path를 list 형식으로 변경
        # path = path if type(path) == list else path.split(separator) if type(path) == str else None ## NOTE: path를 list 형식으로 변경
        if not path:
            return "root"   
        parents_id = "root" if path[0].lower() in ["", "내 드라이브", "my drive", "mydrive", "drive"] else self.id_by_name(path[0])
        for name in path[1:]:
            parents_id = self._child_id_by_name(parents_id, name)
            print(f"name: {name}, parents_id: {parents_id}")
            if parents_id == None:
                break
        return parents_id


    def path_by_id(self, file_id, root=""):
        path = [self.name_by_id(file_id)]
        print(f"path: {path}")
        while file_id != "root":
            file_id = self._parents_id_by_file_id(file_id)
            if file_id == "root":
                break
            file_name = self.name_by_id(file_id)
            path.append(file_name)
        return "/".join(reversed(path))



# main(`__init__`)

In [ ]:
##@@@ Main Function
##============================================================
if __name__ == "__main__":
    user_nick = "satsbymoon"
    gds = GoogleAPI("sheets", user_nick=user_nick, bot_nick=None)
    # # user_nick = "master"
    # # bot_nick = "mats"

    l = gds.list()
    # l = gss.id_by_name(name="Master_Configs_Google", app="sheets")
    pprint(l)

    # 
    parents_id = "1yth9jH4WSenKOBFmlivLXlWcxDnr79i1"
    parents_id = "1ywx8F3Ujx0wH093wznXf0qgYo1mhKDRs"
    # r = gds._children_by_parents_id(parents_id)
    # r = gds._child_id_by_name(parents_id, "GoogleDrive.ipynb")
    # r = gds._ids_by_path(path="/__COLAB/__GOOGLE1/GoogleDrive.ipynb")
    r = gds._ids_by_path(path="/__COLAB/__GOOGLE/")

## list

In [ ]:
    ##@@ NOTE: drive
    # l = gds._list(queries=["parents = '1cXwU3nf9_CldBFWzb8DzJqlgs3sDyc7P'"], out_type="list")


    # path = "/__COLAB/CONFIGS/GOOGLE"
    # path = "/__COLAB/__GOOGLE/GoogeSheets.ipynb"
    # r = gds.id_by_path(path=path)
    # print(r)    
    # gss = Google("sheets", user_nick=None, bot_nick=bot_nick)

## id, name, path

In [ ]:

    # id_by_name
    # r = gds.id_by_name("Master_Configs_Google")
    # _parents_id_by_file_id
    # file_id = "1jVOmj_-RTjv1Pec28WCxpn9c3oJm_cjn"
    # r = gds._parents_id_by_file_id(file_id=file_id)
    # r = gds.path_by_id(file_id=file_id)
    
    # file_id = "1ywx8F3Ujx0wH093wznXf0qgYo1mhKDRs"
    # r = gds.drive.files().get(fileId=file_id, fields="name").execute()["name"]
    # r = gds.name_by_id(file_id)
    pprint(r)


# PlayGround